In [17]:
# CÉLULA DE RECUPERAÇÃO COMPLETA - Recria tudo do zero
print("🔄 RECUPERAÇÃO COMPLETA DO SISTEMA RAG")
print("="*60)

# ========== PASSO 1: VERIFICAR BIBLIOTECAS ==========
print("1️⃣ Verificando bibliotecas...")
try:
    import sys
    from sentence_transformers import SentenceTransformer
    import chromadb
    from chromadb.config import Settings
    from langchain.document_loaders import PyPDFLoader, TextLoader
    from langchain.text_splitter import RecursiveCharacterTextSplitter
    from pathlib import Path
    print("✅ Todas as bibliotecas importadas com sucesso")
except ImportError as e:
    print(f"❌ Erro de importação: {e}")
    exit()

# ========== PASSO 2: CARREGAR MODELO DE EMBEDDING ==========
print("\n2️⃣ Carregando modelo de embedding...")
embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
print("✅ Modelo carregado")

# ========== PASSO 3: RECARREGAR DOCUMENTOS ==========
print("\n3️⃣ Recarregando documentos da pasta data...")
data_path = Path("data")
documentos_reais = []
metadados_docs = []

if data_path.exists():
    # Carregar TXTs
    for arquivo_txt in data_path.glob("*.txt"):
        try:
            loader = TextLoader(str(arquivo_txt), encoding='utf-8')
            docs = loader.load()
            for doc in docs:
                documentos_reais.append(doc.page_content)
                metadados_docs.append({
                    'source': arquivo_txt.name,
                    'type': 'TXT',
                    'size': len(doc.page_content)
                })
            print(f"   ✅ {arquivo_txt.name}: {len(docs[0].page_content)} chars")
        except Exception as e:
            print(f"   ⚠️ Erro em {arquivo_txt.name}: {e}")
    
    # Carregar PDFs
    for arquivo_pdf in data_path.glob("*.pdf"):
        try:
            loader = PyPDFLoader(str(arquivo_pdf))
            docs = loader.load()
            texto_completo = "\n\n".join([doc.page_content for doc in docs])
            documentos_reais.append(texto_completo)
            metadados_docs.append({
                'source': arquivo_pdf.name,
                'type': 'PDF',
                'pages': len(docs),
                'size': len(texto_completo)
            })
            print(f"   ✅ {arquivo_pdf.name}: {len(docs)} páginas")
        except Exception as e:
            print(f"   ⚠️ Erro em {arquivo_pdf.name}: {e}")

print(f"✅ {len(documentos_reais)} documentos carregados")

# ========== PASSO 4: CRIAR CHUNKS ==========
print("\n4️⃣ Criando chunks...")
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
    separators=["\n\n", "\n", ". ", " ", ""]
)

chunks_reais = []
for i, (doc_text, metadata) in enumerate(zip(documentos_reais, metadados_docs)):
    doc_chunks = text_splitter.split_text(doc_text)
    for j, chunk in enumerate(doc_chunks):
        chunk_info = {
            'id': f'doc_{i}_chunk_{j}',
            'text': chunk,
            'source_file': metadata['source'],
            'source_type': metadata['type'],
            'chunk_number': j,
            'char_count': len(chunk)
        }
        chunks_reais.append(chunk_info)

print(f"✅ {len(chunks_reais)} chunks criados")

# ========== PASSO 5: CRIAR BANCO VETORIAL ==========
print("\n5️⃣ Criando banco vetorial...")
client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    allow_reset=True
))

# Limpar collection anterior
try:
    client.delete_collection("meus_documentos_rag")
except:
    pass

collection = client.create_collection("meus_documentos_rag")

# Preparar dados
textos_chunks = [chunk['text'] for chunk in chunks_reais]
ids_chunks = [f"chunk_{i}" for i in range(len(chunks_reais))]

# Gerar embeddings
print("   Gerando embeddings...")
embeddings_chunks = embedding_model.encode(textos_chunks)

# Indexar
print("   Indexando no banco...")
collection.add(
    embeddings=embeddings_chunks.tolist(),
    documents=textos_chunks,
    ids=ids_chunks
)

total_items = collection.count()
print(f"✅ {total_items} chunks indexados no banco vetorial")

# ========== VERIFICAÇÃO FINAL ==========
print("\n" + "="*60)
print("🎉 SISTEMA RAG COMPLETAMENTE RESTAURADO!")
print(f"   📚 Documentos: {len(documentos_reais)}")
print(f"   🧩 Chunks: {len(chunks_reais)}")
print(f"   💾 Banco vetorial: {total_items} itens")
print(f"   🧠 Modelo embedding: all-MiniLM-L6-v2")

# Teste rápido
print("\n🧪 Teste rápido de busca:")
resultado_teste = collection.query(
    query_texts=["inteligência artificial"],
    n_results=1
)
print("✅ Busca funcionando perfeitamente!")

print("\n🚀 Agora você pode fazer buscas semânticas!")

🔄 RECUPERAÇÃO COMPLETA DO SISTEMA RAG
1️⃣ Verificando bibliotecas...
✅ Todas as bibliotecas importadas com sucesso

2️⃣ Carregando modelo de embedding...


incorrect startxref pointer(1)
parsing for Object Streams


✅ Modelo carregado

3️⃣ Recarregando documentos da pasta data...
   ✅ background.txt: 2079 chars
   ✅ coddigo.txt: 59433 chars
   ✅ 2506.11928v1.pdf: 71 páginas
   ✅ Facial_Emotion_Recognition_(FER)_Advances_and_Applications_(2022-2025).pdf: 5 páginas
✅ 4 documentos carregados

4️⃣ Criando chunks...
✅ 325 chunks criados

5️⃣ Criando banco vetorial...
   Gerando embeddings...
   Indexando no banco...
✅ 325 chunks indexados no banco vetorial

🎉 SISTEMA RAG COMPLETAMENTE RESTAURADO!
   📚 Documentos: 4
   🧩 Chunks: 325
   💾 Banco vetorial: 325 itens
   🧠 Modelo embedding: all-MiniLM-L6-v2

🧪 Teste rápido de busca:
✅ Busca funcionando perfeitamente!

🚀 Agora você pode fazer buscas semânticas!


In [18]:
# Célula 1: Verificação basica e importações 
import sys
print(f"Phyton: {sys.version}")

# Verifica se estamos no ambiente correto
print("Verificando se as bibliotecas estão disponiveis...")

try:
    import langchain
    print(f"✅ LangChain: {langchain.__version__}")
except ImportError:
    print("❌ LangChain não encontrado")

try:
    import chromadb
    print(f"✅ ChromaDB: {chromadb.__version__}")
except ImportError:
    print("❌ ChromaDB não encontrado")

try:
    import sentence_transformers
    print(f"✅ Sentence Transformers: {sentence_transformers.__version__}")
except ImportError:
    print("❌ Sentence Transformers não encontrado")

Phyton: 3.13.3 (tags/v3.13.3:6280bb5, Apr  8 2025, 14:47:33) [MSC v.1943 64 bit (AMD64)]
Verificando se as bibliotecas estão disponiveis...
✅ LangChain: 0.3.26
✅ ChromaDB: 1.0.15
✅ Sentence Transformers: 5.0.0


In [19]:
# Célula 2: Experimento com embeddings
from sentence_transformers import SentenceTransformer

# Carregar um modelo pequeno para entender embeddings
print("Carregando modelo de embedding...")
embedding_model = SentenceTransformer("all-MiniLM-L6-v2")
print("✅ Modelo Carregado!")

# Teste básico com frases similares e diferentes
textos = [
    "O gato subiu no telhado",
    "Um felino escalou o teto de casa",
    "Hoje está chovendo muito",
    "Tempestade ocorreu com intensidade",
    "O cachorro late no quintal",
    "O canino ladra mas não morde"
]

print(f"\nVamos analisar {len(textos)} frases:")
for i, texto in enumerate(textos):
    print(f"{i+1}. '{texto}'")

# Gerar embeddings
print("\nGerando embeddings...")
embeddings = embedding_model.encode(textos)
print(f"✅ Shape dos embeddings: {embeddings.shape}")
print(f"✅ Cada texto virou um vetor de {embeddings.shape[1]} dimensões")

Carregando modelo de embedding...
✅ Modelo Carregado!

Vamos analisar 6 frases:
1. 'O gato subiu no telhado'
2. 'Um felino escalou o teto de casa'
3. 'Hoje está chovendo muito'
4. 'Tempestade ocorreu com intensidade'
5. 'O cachorro late no quintal'
6. 'O canino ladra mas não morde'

Gerando embeddings...
✅ Shape dos embeddings: (6, 384)
✅ Cada texto virou um vetor de 384 dimensões


In [20]:
# Célula 3: Calculando a similaridade entre textos
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

print("🔍 Analisando a similaridade entre os nossos 6 textos:")
print()

# Relembrar quais são os textos
for i, texto in enumerate(textos):
    print(f"{i+1}. '{texto}'")

print("\n" + "="*60)

# Calcular similaridade entre todos os textos
similarities = cosine_similarity(embeddings)

print("📊 Matriz de similaridade:")
print("(1.0 = idêntico, 0.0 = neutro, -1.0 = oposto)")
print()

# Mostarr comparação de forma organizada
for i, texto1 in enumerate(textos):
    for j, texto2 in enumerate(textos):
        if i < j: # evita repetições (só mostra a metade da matriz)
            sim = similarities[i][j]
            print(f"Texto {i+1} vs Texto {j+1}: {sim:.3f}")
            print(f"  '{texto1[:30]}...' ↔ '{texto2[:30]}...'")
        
            # Interpretação humana
            if sim > 0.7:
                print(f"  💚 Muito similares!")
            elif sim > 0.4:
                print(f"  💛 Moderadamente similares")
            else:
                print(f"  ❤️ Pouco similares")
            print()

🔍 Analisando a similaridade entre os nossos 6 textos:

1. 'O gato subiu no telhado'
2. 'Um felino escalou o teto de casa'
3. 'Hoje está chovendo muito'
4. 'Tempestade ocorreu com intensidade'
5. 'O cachorro late no quintal'
6. 'O canino ladra mas não morde'

📊 Matriz de similaridade:
(1.0 = idêntico, 0.0 = neutro, -1.0 = oposto)

Texto 1 vs Texto 2: 0.497
  'O gato subiu no telhado...' ↔ 'Um felino escalou o teto de ca...'
  💛 Moderadamente similares

Texto 1 vs Texto 3: 0.469
  'O gato subiu no telhado...' ↔ 'Hoje está chovendo muito...'
  💛 Moderadamente similares

Texto 1 vs Texto 4: 0.418
  'O gato subiu no telhado...' ↔ 'Tempestade ocorreu com intensi...'
  💛 Moderadamente similares

Texto 1 vs Texto 5: 0.548
  'O gato subiu no telhado...' ↔ 'O cachorro late no quintal...'
  💛 Moderadamente similares

Texto 1 vs Texto 6: 0.503
  'O gato subiu no telhado...' ↔ 'O canino ladra mas não morde...'
  💛 Moderadamente similares

Texto 2 vs Texto 3: 0.456
  'Um felino escalou o teto de ca.

In [21]:
# Célula 4: Criando nossa "base de conhecimento"
documentos = [
    """
    Inteligência Artificial (IA) é um campo da ciência da computação que se concentra 
    no desenvolvimento de sistemas capazes de realizar tarefas que normalmente 
    requerem inteligência humana. Isso inclui aprendizado, raciocínio, 
    percepção e tomada de decisão. A IA pode ser classificada em IA fraca 
    (sistemas específicos) e IA forte (inteligência geral).
    """,

    """
    Machine Learning é uma subcategoria da IA que permite que os computadores 
    aprendam e melhorem automaticamente a partir da experiência, sem serem 
    explicitamente programados. Utiliza algoritmos estatísticos para 
    identificar padrões em dados. Os principais tipos são: supervisionado, 
    não supervisionado e por reforço.
    """,

    """
     Python é uma linguagem de programação de alto nível, interpretada e 
    de propósito geral. É amplamente utilizada em ciência de dados, 
    desenvolvimento web, automação e inteligência artificial devido 
    à sua sintaxe simples e rica biblioteca de pacotes como NumPy, 
    Pandas e TensorFlow.
    """,

    """
    LangChain é um framework para desenvolver aplicações com modelos de 
    linguagem. Ele fornece ferramentas para conectar LLMs com fontes de 
    dados externas, criar pipelines de processamento e construir 
    aplicações inteligentes como chatbots e sistemas de pergunta-resposta.
    LangChain facilita a implementação de RAG.
    """,

    """
    Bancos vetoriais são sistemas de banco de dados especializados em 
    armazenar e buscar vetores de alta dimensionalidade. Eles são 
    fundamentais para aplicações de IA que utilizam embeddings, como 
    sistemas de recomendação, busca semântica e RAG. Exemplos incluem 
    ChromaDB, Pinecone e Weaviate.
    """
]

print("📚 Base de Conhecimetno Criada!")
print("="*50)

for i, doc in enumerate(documentos):
    palavras = len(doc.split())
    caracteres = len(doc.strip())
    print(f"Documento {i+1}:")
    print(f"  📄 {palavras} palavras, {caracteres} caracteres")
    print(f"  📝 Tema: {doc.strip()[:50]}...")
    print()

print(f"✅ Total: {len(documentos)} documentos em nossa base")
    

📚 Base de Conhecimetno Criada!
Documento 1:
  📄 50 palavras, 369 caracteres
  📝 Tema: Inteligência Artificial (IA) é um campo da ciência...

Documento 2:
  📄 42 palavras, 334 caracteres
  📝 Tema: Machine Learning é uma subcategoria da IA que perm...

Documento 3:
  📄 42 palavras, 299 caracteres
  📝 Tema: Python é uma linguagem de programação de alto níve...

Documento 4:
  📄 42 palavras, 329 caracteres
  📝 Tema: LangChain é um framework para desenvolver aplicaçõ...

Documento 5:
  📄 41 palavras, 309 caracteres
  📝 Tema: Bancos vetoriais são sistemas de banco de dados es...

✅ Total: 5 documentos em nossa base


In [22]:
# Célula 5: Entendendo Chunking (Divisão de Texto)
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("🔪 Aprendendo sobre Chunking")
print("="*40)

# Configurar o divisor de texto
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=200, # Tamanho máximo de cada pedaço de texto (em caracteres)
    chunk_overlap=50, # Sobreposição entre pedaços
    separators=["\n\n", "\n", ". ", " ", ""] # Como dividir (ordem de preferência)
)

print("⚙️ Configuração do Text Splitter:")
print(f"   📏 Tamanho máximo do chunk: {text_splitter._chunk_size} caracteres")
print(f"   🔗 Sobreposição: {text_splitter._chunk_overlap} caracteres")
print(f"   ✂️ Separadores: {text_splitter._separators}")
print()

# Dividir nossos docuemntos
chunks = []
chunk_counter = 0

for i, doc in enumerate(documentos):
    doc_limpo = doc.strip()
    doc_chunks = text_splitter.split_text(doc_limpo)

    print(f"📄 Documento {i+1} → {len(doc_chunks)} chunk(s)")
    
    for j, chunk in enumerate(doc_chunks):
         chunk_info = {
        'id': f"chunk_{chunk_counter}",
        'text': chunk,
        'source': f"documento_{i+1}",
        'chunk_num': j,
        'chars': len(chunk)
    }
    chunks.append(chunk_info)
    
    print(f" Chunk {j+1}: {len(chunk)} chars → '{chunk[:60]}...'")
    chunk_counter += 1
print()

print("="*50)
print(f"🎯 Resultado Final:")
print(f"   📚 {len(documentos)} documentos originais")
print(f"   🧩 {len(chunks)} chunks criados")
print(f"   📊 Média de {len(chunks)/len(documentos):.1f} chunks por documento")

🔪 Aprendendo sobre Chunking
⚙️ Configuração do Text Splitter:
   📏 Tamanho máximo do chunk: 200 caracteres
   🔗 Sobreposição: 50 caracteres
   ✂️ Separadores: ['\n\n', '\n', '. ', ' ', '']

📄 Documento 1 → 3 chunk(s)
 Chunk 3: 55 chars → '(sistemas específicos) e IA forte (inteligência geral)....'
📄 Documento 2 → 2 chunk(s)
 Chunk 2: 179 chars → 'explicitamente programados. Utiliza algoritmos estatísticos ...'
📄 Documento 3 → 2 chunk(s)
 Chunk 2: 157 chars → 'desenvolvimento web, automação e inteligência artificial dev...'
📄 Documento 4 → 2 chunk(s)
 Chunk 2: 183 chars → 'dados externas, criar pipelines de processamento e construir...'
📄 Documento 5 → 2 chunk(s)
 Chunk 2: 171 chars → 'fundamentais para aplicações de IA que utilizam embeddings, ...'

🎯 Resultado Final:
   📚 5 documentos originais
   🧩 5 chunks criados
   📊 Média de 1.0 chunks por documento


In [23]:
# Célula 6: Explorando arquivos reais na pasta data
import os
from pathlib import Path

# Vrificar pasta data
data_path = Path("data")

if data_path.exists():
    print("📁 Arquivos encontrados na pasta 'data':")
    print("="*50)

    arquivo_pdf = []
    arquivo_txt = []
    outros_arquivos = []

    for arquivo in data_path.iterdir():
        if arquivo.is_file():
            tamanho_kb = arquivo.stat().st_size / 1024

            if arquivo.suffix.lower() == '.pdf':
                arquivo_pdf.append(arquivo)
                print(f"📄 PDF: {arquivo.name} ({tamanho_kb:.1f} KB)")
            elif arquivo.suffix.lower() == '.txt':
                arquivo_txt.append(arquivo)
                print(f"📝 TXT: {arquivo.name} ({tamanho_kb:.1f} KB)")
            else:
                outros_arquivos.append(arquivo)
                print(f"❓ Outro: {arquivo.name} ({tamanho_kb:.1f} KB)")
                
    print(f"\n📊 Resumo:")
    print(f"   📄 {len(arquivo_pdf)} arquivo(s) PDF")
    print(f"   📝 {len(arquivo_txt)} arquivo(s) TXT")
    print(f"   ❓ {len(outros_arquivos)} outro(s) arquivo(s)")

    if len(arquivo_pdf) + len(arquivo_txt) > 0:
        print("\n✅ Vamos usar esses arquivos no nosso RAG!")
    else:
        print("\n⚠️ Nenhum PDF ou TXT encontrado. Vamos usar documentos sintéticos.")
else:
    print("❌ Pasta 'data' não encontrada.")
    print("💡 Crie a pasta 'data' e coloque seus arquivos lá, ou vamos continuar com documentos sintéticos.")

📁 Arquivos encontrados na pasta 'data':
📄 PDF: 2506.11928v1.pdf (712.0 KB)
📝 TXT: background.txt (2.1 KB)
📝 TXT: coddigo.txt (60.7 KB)
📄 PDF: Facial_Emotion_Recognition_(FER)_Advances_and_Applications_(2022-2025).pdf (78.3 KB)
❓ Outro: imagem (8).png (38.0 KB)

📊 Resumo:
   📄 2 arquivo(s) PDF
   📝 2 arquivo(s) TXT
   ❓ 1 outro(s) arquivo(s)

✅ Vamos usar esses arquivos no nosso RAG!


In [24]:
# Célula 7: Carergando documentos reais
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.schema import Document

print("📚 Carregando documentos reais...")
print("="*40)

documentos_reais = []
metadados_docs = []

# Carregar arquivos TXT
if arquivo_txt:
    print("📝 Carregando arquivos TXT:")
    for arquivos_txt in arquivo_txt:
        try:
            loader = TextLoader(str(arquivos_txt), encoding='utf-8')
            docs = loader.load()

            for doc in docs:
                documentos_reais.append(doc.page_content)
                metadados_docs.append({
                    'source': arquivos_txt.name,
                    'type': 'TXT',
                    'size': len(doc.page_content)
                })
                
            print(f"   ✅ {arquivo_txt.name}: {len(docs[0].page_content)} caracteres")
            
        except Exception as e:
            print(f"   ❌ Erro ao carregar {arquivos_txt.name}: {e}")

# Carregar arquivo PDF
if arquivo_pdf:
    print("📝 Carregando arquivos PDF:")
    for arquivos_pdf in arquivo_pdf:
        try:
            loader = PyPDFLoader(str(arquivos_pdf))
            docs = loader.load()

            # Juntar todas as paginas do PDF
            texto_completo = "\n\n".join([doc.page_content for doc in docs])
            documentos_reais.append(texto_completo)

            metadados_docs.append({
                'source': arquivo_pdf.name,
                'type': 'PDF',
                'pages': len(docs),
                'size': len(texto_completo)
            })

            print(f"   ✅ {arquivo_pdf.name}: {len(docs)} páginas, {len(texto_completo)} caracteres")
        except Exception as e:
            print(f"   ❌ Erro ao carregar {arquivos_pdf.name}: {e}")

# Resultado final
print("\n" + "="*50)
print(f"🎯 Documentos carregados com sucesso:")
print(f"   📄 Total: {len(documentos_reais)} documento(s)")

for i, metadata in enumerate(metadados_docs):
    print(f"   {i+1}. {metadata['source']} ({metadata['type']}) - {metadata['size']} chars")

# Se não conseguiu carregar nenhum, usar sintéticos
if not documentos_reais:
    print("\n⚠️ Nenhum documento foi carregado. Usando documentos sintéticos...")
    # (aqui poderia voltar aos documentos sintéticos se necessário)
else:
    print(f"\n✅ Vamos usar seus {len(documentos_reais)} documento(s) real(is) no RAG!")
    
    # Mostrar uma amostra do primeiro documento
    print(f"\n📄 Amostra do primeiro documento ({metadados_docs[0]['source']}):")
    print(f"'{documentos_reais[0][:200]}...'")

incorrect startxref pointer(1)
parsing for Object Streams


📚 Carregando documentos reais...
📝 Carregando arquivos TXT:
   ❌ Erro ao carregar background.txt: 'list' object has no attribute 'name'
   ❌ Erro ao carregar coddigo.txt: 'list' object has no attribute 'name'
📝 Carregando arquivos PDF:
   ❌ Erro ao carregar 2506.11928v1.pdf: 'list' object has no attribute 'name'
   ❌ Erro ao carregar Facial_Emotion_Recognition_(FER)_Advances_and_Applications_(2022-2025).pdf: 'list' object has no attribute 'name'

🎯 Documentos carregados com sucesso:
   📄 Total: 4 documento(s)
   1. background.txt (TXT) - 2079 chars
   2. coddigo.txt (TXT) - 59433 chars

✅ Vamos usar seus 4 documento(s) real(is) no RAG!

📄 Amostra do primeiro documento (background.txt):
'já trabalhei com Llama inclusive tenho participação da primeira equipe de IA em uma competição no Kaggle, trabalho diariamente com OpenAI, Gemini e Claude da Anthopric.
Ainda não tive oportunidade de ...'


In [25]:
# Célula 8: Aplicando chunking aos documentos reais
from langchain.text_splitter import RecursiveCharacterTextSplitter

print("🔪 Aplicando Chunking aos Documentos Reais")
print("="*50)

# Configurar splitter para documentos reais (chunks maiores)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, # Maior para documentos reais
    chunk_overlap = 200, # Overlap mairo para preservar contexto
    separators = ["\n\n", ". ", " ", ""]
)

print("⚙️ Configuração do Chunking:")
print(f"   📏 Tamanho máximo: {text_splitter._chunk_size} caracteres")
print(f"   🔗 Sobreposição: {text_splitter._chunk_overlap} caracteres")
print()

# Processar cada documento
chunk_reais = []
estatisticas = []

for i, (doc_text, metadata) in enumerate(zip(documentos_reais, metadados_docs)):
    print(f"📄 Processando: {metadata['source']}")
    print(f"   📊 Documento original: {len(doc_text):,} caracteres")

    # Aplicar chunking
    doc_chunks = text_splitter.split_text(doc_text)

    print(f"   🧩 Resultado: {len(doc_chunks)} chunks")

    # Armazenar chunks com metadados
    for j, chunk in enumerate(doc_chunks):
        chunk_info = {
            'id': f'doc_{i}_chunk_{j}',
            'text': chunk,
            'source_file': metadata['source'],
            'source_type': metadata['type'],
            'chunk_number': j,
            'total_chunks': len(doc_chunks),
            'char_count': len(chunk)
        }
        chunk_reais.append(chunk_info)

    # Estatisticas para documento
    tamanhos_chunks = [len(chunk) for chunk in doc_chunks]
    stats = {
        'arquivo': metadata['source'],
        'chars_original': len(doc_text),
        'num_chunks': len(doc_chunks),
        'chunk_min': min(tamanhos_chunks),
        'chunk_max': max(tamanhos_chunks),
        'chunk_medio': sum(tamanhos_chunks) / len(tamanhos_chunks)
    }
    estatisticas.append(stats)

    print(f"   📈 Tamanho dos chunks: {stats['chunk_min']}-{stats['chunk_max']} chars (média: {stats['chunk_medio']:.0f})")

    # Mostrar amostra do primeiro chunk
    print(f"   📝 Primeiro chunk: '{doc_chunks[0][:100]}...'")
    print()

# Resumo final
print("="*60)
print("🎯 Resumo Final do Chunking:")
print(f"   📚 {len(documentos_reais)} documentos processados")
print(f"   🧩 {len(chunk_reais)} chunks criados no total")
print(f"   📊 Média de {len(chunk_reais)/len(documentos_reais):.1f} chunks por documento")

print("\n📋 Detalhes por arquivo:")
for stat in estatisticas:
    print(f"   📄 {stat['arquivo']}: {stat['num_chunks']} chunks (média {stat['chunk_medio']:.0f} chars)")

# Encontrar o documento com mais chunks
doc_mais_chunks = max(estatisticas, key=lambda x: x['num_chunks'])
print(f"\n🏆 Documento com mais chunks: {doc_mais_chunks['arquivo']} ({doc_mais_chunks['num_chunks']} chunks)")

🔪 Aplicando Chunking aos Documentos Reais
⚙️ Configuração do Chunking:
   📏 Tamanho máximo: 1000 caracteres
   🔗 Sobreposição: 200 caracteres

📄 Processando: background.txt
   📊 Documento original: 2,079 caracteres
   🧩 Resultado: 4 chunks
   📈 Tamanho dos chunks: 354-997 chars (média: 568)
   📝 Primeiro chunk: 'já trabalhei com Llama inclusive tenho participação da primeira equipe de IA em uma competição no Ka...'

📄 Processando: coddigo.txt
   📊 Documento original: 59,433 caracteres
   🧩 Resultado: 71 chunks
   📈 Tamanho dos chunks: 355-998 chars (média: 894)
   📝 Primeiro chunk: '// >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<...'

🎯 Resumo Final do Chunking:
   📚 4 documentos processados
   🧩 75 chunks criados no total
   📊 Média de 18.8 chunks por documento

📋 Detalhes por arquivo:
   📄 background.txt: 4 chunks (média 568 chars)
   📄 coddigo.txt: 71 chunks (média 894 chars)

🏆 Documento com mais chunks: coddigo.txt (71 chunks)


In [26]:
# Célula 9: Banco vetorial simplificado (sem metadados complexos)
import chromadb
from chromadb.config import Settings
import time

print("🏗️ Criando Banco Vetorial - Versão Simplificada")
print("="*60)

# Verificar dados antes de processar
print("🔍 Verificando dados antes do processamento:")
print(f"   📚 documentos_reais: {len(documentos_reais)} itens")
print(f"   🧩 chunks_reais: {len(chunks_reais)} itens")

# Mostrar estrutura do primeiro chunk
if len(chunks_reais) > 0:
    primeiro_chunk = chunks_reais[0]
    print(f"   📄 Estrutura do primeiro chunk:")
    for key, value in primeiro_chunk.items():
        print(f"      {key}: {type(value)} = {str(value)[:50]}...")
else:
    print("   ❌ Nenhum chunk encontrado!")
    exit()

# Configurar ChromaDB
client = chromadb.Client(Settings(
    anonymized_telemetry=False,
    allow_reset=True
))

# Limpar e criar coleção
collection_name = "meus_documentos_rag"
try:
    client.delete_collection(collection_name)
except:
    pass

collection = client.create_collection(name=collection_name)
print(f"✅ Coleção '{collection_name}' criada")

# Extrair dados dos chunks
textos_chunks = []
ids_chunks = []
metadados_simples = []

for i, chunk in enumerate(chunks_reais):
    # Garantir que temos texto
    if 'text' in chunk and chunk['text'].strip():
        textos_chunks.append(chunk['text'])
        ids_chunks.append(f"chunk_{i}")  # ID simples
        
        # Metadados muito simples (apenas arquivo fonte)
        if 'source_file' in chunk:
            metadados_simples.append({'source': chunk['source_file']})
        else:
            metadados_simples.append({'source': 'unknown'})

print(f"\n📊 Dados preparados:")
print(f"   📝 {len(textos_chunks)} textos válidos")
print(f"   🏷️ {len(ids_chunks)} IDs criados") 
print(f"   📋 {len(metadados_simples)} metadados criados")

# Verificar se temos dados válidos
if len(textos_chunks) == 0:
    print("❌ Nenhum texto válido encontrado!")
    exit()

# Gerar embeddings
print(f"\n🧠 Gerando embeddings...")
start_time = time.time()
embeddings_chunks = embedding_model.encode(textos_chunks)
embedding_time = time.time() - start_time

print(f"✅ Embeddings gerados:")
print(f"   ⏱️ Tempo: {embedding_time:.2f} segundos")
print(f"   📊 Shape: {embeddings_chunks.shape}")

# Adicionar ao ChromaDB
print(f"\n💾 Adicionando ao banco vetorial...")
try:
    collection.add(
        embeddings=embeddings_chunks.tolist(),
        documents=textos_chunks,
        metadatas=metadados_simples,
        ids=ids_chunks
    )
    
    total_items = collection.count()
    print(f"✅ Sucesso! {total_items} chunks indexados")
    
    # Testar uma busca rápida
    print(f"\n🧪 Teste rápido de busca:")
    resultados_teste = collection.query(
        query_texts=["inteligência artificial"],
        n_results=3
    )
    print(f"   ✅ Busca funcionando: {len(resultados_teste['documents'][0])} resultados encontrados")
    
except Exception as e:
    print(f"❌ Erro ao adicionar: {e}")
    print("💡 Vamos tentar sem metadados...")
    
    # Tentar sem metadados
    try:
        collection.add(
            embeddings=embeddings_chunks.tolist(),
            documents=textos_chunks,
            ids=ids_chunks
            # Sem metadatas
        )
        
        total_items = collection.count()
        print(f"✅ Sucesso sem metadados! {total_items} chunks indexados")
        
    except Exception as e2:
        print(f"❌ Erro mesmo sem metadados: {e2}")

print(f"\n🎉 Banco vetorial está pronto!")

🏗️ Criando Banco Vetorial - Versão Simplificada
🔍 Verificando dados antes do processamento:
   📚 documentos_reais: 4 itens
   🧩 chunks_reais: 325 itens
   📄 Estrutura do primeiro chunk:
      id: <class 'str'> = doc_0_chunk_0...
      text: <class 'str'> = já trabalhei com Llama inclusive tenho participaçã...
      source_file: <class 'str'> = background.txt...
      source_type: <class 'str'> = TXT...
      chunk_number: <class 'int'> = 0...
      char_count: <class 'int'> = 969...
✅ Coleção 'meus_documentos_rag' criada

📊 Dados preparados:
   📝 325 textos válidos
   🏷️ 325 IDs criados
   📋 325 metadados criados

🧠 Gerando embeddings...
✅ Embeddings gerados:
   ⏱️ Tempo: 13.01 segundos
   📊 Shape: (325, 384)

💾 Adicionando ao banco vetorial...
✅ Sucesso! 325 chunks indexados

🧪 Teste rápido de busca:
   ✅ Busca funcionando: 3 resultados encontrados

🎉 Banco vetorial está pronto!


In [33]:
# Célula 10: Buscas semânticas - VERSÃO CORRIGIDA
print("🔍 Primeira Busca Semântica Real")
print("="*50)

# Verificar se temos o banco vetorial
try:
    total_items = collection.count()
    print(f"✅ Banco vetorial disponível: {total_items} chunks")
except:
    print("❌ Banco vetorial não encontrado!")
    exit()

# Função corrigida para fazer buscas
def buscar_documentos(pergunta, num_resultados=3):
    print(f"\n❓ Pergunta: '{pergunta}'")
    print("-" * 60)
    
    try:
        # Fazer a busca
        resultados = collection.query(
            query_texts=[pergunta],
            n_results=num_resultados
        )
        
        # Debug: mostrar estrutura do resultado
        print(f"🔍 Debug - Chaves do resultado: {list(resultados.keys())}")
        
        # Extrair informações de forma segura
        documentos_encontrados = resultados.get('documents', [[]])[0]
        ids = resultados.get('ids', [[]])[0]
        
        # ChromaDB pode retornar 'distances' ou não, vamos verificar
        if 'distances' in resultados:
            distancias = resultados['distances'][0]
        else:
            # Se não tem distances, simular com valores neutros
            distancias = [0.5] * len(documentos_encontrados)
            print("⚠️ Distances não disponíveis, usando valores simulados")
        
        # Verificar se temos resultados
        if not documentos_encontrados:
            print("❌ Nenhum resultado encontrado!")
            return [], []
        
        # Mostrar resultados
        for i, (doc, distancia, id_chunk) in enumerate(zip(documentos_encontrados, distancias, ids)):
            similaridade = 1 - distancia  # Converter distância em similaridade
            print(f"📄 Resultado {i+1} (ID: {id_chunk}):")
            print(f"   🎯 Similaridade: {similaridade:.3f} (quanto maior, mais similar)")
            print(f"   📝 Texto encontrado:")
            print(f"      '{doc[:300]}...'")
            print()
        
        return documentos_encontrados, distancias
        
    except Exception as e:
        print(f"❌ Erro na busca: {e}")
        return [], []

# Teste 1: Buscar informações sobre experiência profissional
print("🧪 TESTE 1: Experiência Profissional")
docs1, dist1 = buscar_documentos("experiência com inteligência artificial e machine learning")

# Teste 2: Buscar sobre Kaggle (mencionado no seu background)
print("\n🧪 TESTE 2: Competições e Kaggle") 
docs2, dist2 = buscar_documentos("participação em competição Kaggle")

# Teste 3: Buscar sobre código/programação
print("\n🧪 TESTE 3: Código e Programação")
docs3, dist3 = buscar_documentos("código Python machine learning")

# Teste 4: Buscar sobre ferramentas de IA
print("\n🧪 TESTE 4: Ferramentas de IA")
docs4, dist4 = buscar_documentos("OpenAI Gemini Claude LLM")

# Análise de resultados (só se temos dados válidos)
print("\n" + "="*60)
print("📊 ANÁLISE DOS RESULTADOS:")

todos_testes = [
    ("Experiência IA", dist1),
    ("Kaggle", dist2), 
    ("Código Python", dist3),
    ("Ferramentas IA", dist4)
]

similaridades_validas = []
for nome, distancias in todos_testes:
    if distancias:  # Se tem resultados
        melhor_similaridade = 1 - min(distancias)
        similaridades_validas.append(melhor_similaridade)
        print(f"   {nome}: {melhor_similaridade:.3f}")
    else:
        print(f"   {nome}: Sem resultados")

if similaridades_validas:
    melhor_busca = max(similaridades_validas)
    print(f"\n🏆 Melhor match encontrado: {melhor_busca:.3f}")
    
    if melhor_busca > 0.8:
        print("   💚 Excelente! O sistema entendeu muito bem a pergunta")
    elif melhor_busca > 0.6:
        print("   💛 Bom! O sistema encontrou conteúdo relevante")
    else:
        print("   🔍 O sistema encontrou algo, mas pode não ser muito específico")
else:
    print("⚠️ Nenhum resultado válido encontrado")

print("\n✨ Buscas semânticas concluídas!")

🔍 Primeira Busca Semântica Real
✅ Banco vetorial disponível: 325 chunks
🧪 TESTE 1: Experiência Profissional

❓ Pergunta: 'experiência com inteligência artificial e machine learning'
------------------------------------------------------------
🔍 Debug - Chaves do resultado: ['ids', 'embeddings', 'documents', 'uris', 'included', 'data', 'metadatas', 'distances']
📄 Resultado 1 (ID: chunk_323):
   🎯 Similaridade: -0.184 (quanto maior, mais similar)
   📝 Texto encontrado:
      '• Explainable AI (XAI) in FER: Enhancing the interpretability of FER models to 
understand how they arrive at their predictions, which is crucial for building trust and 
addressing ethical concerns.
• Edge AI for FER: Deploying FER models on edge devices for real-time, on-device 
processing, reducin...'

📄 Resultado 2 (ID: chunk_90):
   🎯 Similaridade: -0.347 (quanto maior, mais similar)
   📝 Texto encontrado:
      'the same rules as humans. This pipeline is detailed in Appendix A3.
Difficulty tiers. To streamline 